In [7]:
!pip install openai datasets


  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
timm 1.0.15 requires torchvision, which is not installed.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [8]:
!pip install openai==0.28


In [6]:
!pip install --upgrade openai datasets fsspec==2025.3.0


  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.0/662.0 kB 13.0 MB/s eta 0:00:00
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.76.2
    Uninstalling openai-1.76.2:
      Successfully uninstalled openai-1.76.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.1.0 requires gcsfs>=2023.3.0, which is not installed.
timm 1.0.15 requires torch, which is not installed.
timm 1.0.15 requires torchvision, which is not installed.


In [9]:
!pip install "torch==2.5.0" "nvidia-cudnn-cu12==9.1.0.70" "fsspec==2025.3.2" "gcsfs==2025.3.2"


  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.
timm 1.0.15 requires torchvision, which is not installed.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.


In [10]:
import openai
from datasets import load_dataset
import random


In [ ]:
openai.api_key ="OPEN AI KEY"

In [12]:
def create_prompt(examples, test_utterance):
    prompt = "Classify the dialogue act of each utterance.\n\n"
    for ex in examples:
        prompt += f"Utterance: {ex['text']}\nDialogue Act: {ex['act']}\n\n"
    prompt += f"Utterance: {test_utterance}\nDialogue Act:"
    return prompt

def gpt3_predict(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=10
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("⚠️ OpenAI API Error:", e)
        return "ERROR"

def load_json_dataset(dataset_name):
    import requests
    import json

    base_url = f"https://raw.githubusercontent.com/zihaohe123/speak-turn-emb-dialog-act-clf/master/data/{dataset_name}"
    train_url = f"{base_url}/train.json"
    test_url = f"{base_url}/test.json"

    def load_json_lines(url):
        response = requests.get(url)
        lines = response.text.strip().split('\n')
        return [json.loads(line) for line in lines if line.strip()]

    train_data = load_json_lines(train_url)
    test_data = load_json_lines(test_url)

    return train_data, test_data


def evaluate_dataset(dataset_name, k=5, test_samples=10):
    print(f"\n=== Evaluating {dataset_name.upper()} ===")
    train_data, test_data = load_json_dataset(dataset_name)
    random.seed(42)
    correct = 0

    for i in range(test_samples):
        few_shot_examples = random.sample(train_data, k)
        test_example = test_data[i]
        prompt = create_prompt(few_shot_examples, test_example['text'])
        prediction = gpt3_predict(prompt)
        print(f"[{i+1}] Predicted: {prediction} | Actual: {test_example['act']}")
        if prediction.lower().strip() == test_example['act'].lower().strip():
            correct += 1

    accuracy = correct / test_samples
    print(f"\n✅ Accuracy on {test_samples} samples from {dataset_name.upper()}: {accuracy:.2f}")


In [13]:
def create_prompt(examples, test_utterance):
    prompt = "Classify the dialogue act of each utterance.\n\n"
    for ex in examples:
        prompt += f"Utterance: {ex['text']}\nDialogue Act: {ex['act']}\n\n"
    prompt += f"Utterance: {test_utterance}\nDialogue Act:"
    return prompt

def gpt3_predict(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=10
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("⚠️ OpenAI API Error:", e)
        return "ERROR"



!wget https://github.com/zihaohe123/speak-turn-emb-dialog-act-clf/raw/master/data.zip
!unzip data.zip


import pandas as pd

train_df = pd.read_csv("data/mrda/train.csv")






--2025-05-07 23:56:23--  https://github.com/zihaohe123/speak-turn-emb-dialog-act-clf/raw/master/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zihaohe123/speak-turn-emb-dialog-act-clf/master/data.zip [following]
--2025-05-07 23:56:24--  https://raw.githubusercontent.com/zihaohe123/speak-turn-emb-dialog-act-clf/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7637050 (7.3M) [application/zip]
Saving to: ‘data.zip.2’

data.zip.2          100%[===================>]   7.28M  --.-KB/s    in 0.1s    

2025-05-07 23:56:24 (75.0 MB/s) - ‘data.zip.2’ saved [7637050/7637050]

Archi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.77.0
    Uninstalling openai-1.77.0:
      Successfully uninstalled openai-1.77.0


In [ ]:
import openai
import pandas as pd
import random

# ⛔ Replace with your real OpenAI key
openai.api_key = "OPEN AI KEY"


In [16]:
def create_prompt(examples, test_utterance):
    prompt = "Classify the dialogue act of each utterance.\n\n"
    for ex in examples:
        prompt += f"Utterance: {ex['text']}\nDialogue Act: {ex['act']}\n\n"
    prompt += f"Utterance: {test_utterance}\nDialogue Act:"
    return prompt

def gpt3_predict(prompt):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.0,
            max_tokens=10
        )
        return response['choices'][0]['message']['content'].strip()
    except Exception as e:
        print("⚠️ OpenAI API Error:", e)
        return "ERROR"


In [17]:
def load_dataset_local(dataset_name):
    train_path = f"data/{dataset_name}/train.csv"
    test_path = f"data/{dataset_name}/test.csv"

    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    train_data = train_df.to_dict(orient="records")
    test_data = test_df.to_dict(orient="records")

    print(f"✅ Loaded {len(train_data)} train and {len(test_data)} test samples from {dataset_name.upper()}")
    return train_data, test_data


In [18]:
def evaluate_dataset(dataset_name, k=5, test_samples=10):
    print(f"\n=== Evaluating {dataset_name.upper()} ===")
    train_data, test_data = load_dataset_local(dataset_name)

    correct = 0
    random.seed(42)

    for i in range(test_samples):
        few_shot_examples = random.sample(train_data, k)
        test_example = test_data[i]
        prompt = create_prompt(few_shot_examples, test_example['text'])
        prediction = gpt3_predict(prompt)

        print(f"[{i+1}] Predicted: {prediction} | Actual: {test_example['act']}")
        if str(prediction).lower().strip() == str(test_example['act']).lower().strip():

            correct += 1

    accuracy = correct / test_samples
    print(f"\n✅ Accuracy on {test_samples} samples from {dataset_name.upper()}: {accuracy:.2f}")


In [20]:
evaluate_dataset("mrda", k=5, test_samples=20)
evaluate_dataset("swda", k=5, test_samples=20)
evaluate_dataset("dyda", k=5, test_samples=20)



=== Evaluating MRDA ===
✅ Loaded 75016 train and 15053 test samples from MRDA
[1] Predicted: 2 | Actual: 0
[2] Predicted: 0 | Actual: 0
[3] Predicted: 0 | Actual: 0
[4] Predicted: 1 | Actual: 3
[5] Predicted: 0 | Actual: 0
[6] Predicted: 3 | Actual: 3
[7] Predicted: Statement | Actual: 2
[8] Predicted: Statement | Actual: 0
[9] Predicted: 1 | Actual: 0
[10] Predicted: 1 | Actual: 0
[11] Predicted: 3 | Actual: 0
[12] Predicted: 1 | Actual: 0
[13] Predicted: 0 | Actual: 0
[14] Predicted: 1 | Actual: 0
[15] Predicted: 3 | Actual: 0
[16] Predicted: 1 | Actual: 0
[17] Predicted: 1 | Actual: 2
[18] Predicted: 0 | Actual: 0
[19] Predicted: 3 | Actual: 4
[20] Predicted: 0 | Actual: 0

✅ Accuracy on 20 samples from MRDA: 0.35

=== Evaluating SWDA ===
✅ Loaded 193325 train and 4514 test samples from SWDA
[1] Predicted: 5 | Actual: 8
[2] Predicted: 35 | Actual: 8
[3] Predicted: 8 | Actual: 35
[4] Predicted: 11 | Actual: 6
[5] Predicted: 8 | Actual: 42
[6] Predicted: 9 | Actual: 6
[7] Predicted: 